- https://nijianmo.github.io/amazon/index.html
- https://jmcauley.ucsd.edu/data/amazon/ 
- https://colab.research.google.com/drive/1Zv6MARGQcrBbLHyjPVVMZVnRWsRnVMpV

In [1]:
prfx = 'prep_20210304A1'

# get data

    download from http://localhost:8080/notebooks/git/product-category/notebooks/secrets_get_amazon_data.ipynb

In [2]:
import pandas as pd
import gzip
import json

In [3]:
pdata = "/data/git/product-category/data/amazon_meta_data"
!ls -hl {pdata}
fnms = !ls {pdata}
fnms = [o for o in fnms if o.endswith('.json.gz')]
domains = [o.strip('meta_').strip('.json.gz') for o in fnms]
dmn2fnm = dict(zip(domains, fnms))
dmn2fnm

total 13G
-rw-rw-r-- 1 ubuntu ubuntu   30M Dec  2 00:07 meta_AMAZON_FASHION.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  9.6M Aug  7  2020 meta_All_Beauty.json.gz
-rw-rw-r-- 1 ubuntu ubuntu   57M Aug  7  2020 meta_Appliances.json.gz
-rw-rw-r-- 1 ubuntu ubuntu   57M Aug  7  2020 meta_Appliances.json.gz.1
-rw-rw-r-- 1 ubuntu ubuntu  125M Aug  7  2020 meta_Arts_Crafts_and_Sewing.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.5G Aug  7  2020 meta_Automotive.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.2G Aug  7  2020 meta_Books.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.2G Aug  7  2020 meta_Books.json.gz.1
-rw-rw-r-- 1 ubuntu ubuntu  153M Aug  7  2020 meta_CDs_and_Vinyl.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  333M Aug  7  2020 meta_Cell_Phones_and_Accessories.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.5G Dec  2 00:18 meta_Clothing_Shoes_and_Jewelry.json.gz
-rw-rw-r-- 1 ubuntu ubuntu   12M Aug  7  2020 meta_Digital_Music.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.2G Aug  7  2020 meta_Electronics.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  37

{'AMAZON_FASHION': 'meta_AMAZON_FASHION.json.gz',
 'All_Beauty': 'meta_All_Beauty.json.gz',
 'Appliance': 'meta_Appliances.json.gz',
 'Arts_Crafts_and_Sewi': 'meta_Arts_Crafts_and_Sewing.json.gz',
 'Automotive': 'meta_Automotive.json.gz',
 'Book': 'meta_Books.json.gz',
 'CDs_and_Vinyl': 'meta_CDs_and_Vinyl.json.gz',
 'Cell_Phones_and_Accessorie': 'meta_Cell_Phones_and_Accessories.json.gz',
 'Clothing_Shoes_and_Jewelry': 'meta_Clothing_Shoes_and_Jewelry.json.gz',
 'Digital_Music': 'meta_Digital_Music.json.gz',
 'Electronic': 'meta_Electronics.json.gz',
 'Gift_Card': 'meta_Gift_Cards.json.gz',
 'Grocery_and_Gourmet_Food': 'meta_Grocery_and_Gourmet_Food.json.gz',
 'Home_and_Kitche': 'meta_Home_and_Kitchen.json.gz',
 'Industrial_and_Scientific': 'meta_Industrial_and_Scientific.json.gz',
 'Kindle_Store': 'meta_Kindle_Store.json.gz',
 'Luxury_Beauty': 'meta_Luxury_Beauty.json.gz',
 'Magazine_Subscripti': 'meta_Magazine_Subscriptions.json.gz',
 'Movies_and_TV': 'meta_Movies_and_TV.json.gz',
 

In [4]:
KEYS2USE = set(['category', 'description', 'title', 'brand', 'feature', 'asin'])

In [5]:
# https://colab.research.google.com/drive/1Zv6MARGQcrBbLHyjPVVMZVnRWsRnVMpV#scrollTo=7igYuRaV4bF7
    
### load the meta data
def get_meta_data(domain, nrows=None):
    keys2use = KEYS2USE
    fnm = dmn2fnm[domain]
    data = []
    with gzip.open(f'{pdata}/{fnm}') as f:
        for i,l in enumerate(f):
            dat = json.loads(l.strip())
            dat = {k:v for k,v in dat.items() if k in keys2use}
            data.append(dat)
            if nrows and i>nrows: break

    # total length of list, this number equals total number of products
#     print("len(data)", len(data))

    # convert list into pandas dataframe
    df = pd.DataFrame.from_dict(data)
#     print("len(df)", len(df))

    ### remove rows with unformatted title (i.e. some 'title' may still contain html style content)
    df3 = df.fillna('')
    df4 = df3[df3.title.str.contains('getTime')] # unformatted rows
    df5 = df3[~df3.title.str.contains('getTime')] # filter those unformatted rows
#     print("len(df4)", len(df4))
#     print("len(df5)", len(df5))

    return df5

In [6]:
df = get_meta_data(domains[3], nrows=10000)
df.head(3)

,category,description,title,brand,feature,asin
0,"[Arts, Crafts & Sewing, Sewing, Trim & Embelli...",[The patch features the sweaty masculine hands...,You Son of a Bitch! 1987 Embroidered Patch,Honchosfx,[You son of a bitch patch - exclusive to Honch...,6665560953
1,"[Arts, Crafts & Sewing, Crafting, Paper & Pape...","[With 5 packs stars folding paper, each pack a...",Origami Stars Papers Package 1H (5 packs),,[],7000000376
2,"[Arts, Crafts & Sewing, Painting, Drawing & Ar...",[],Yi De Ge Chinese Calligraphy Sumi Drawing Blac...,MasterChinese,[],7000001089


# preprocess data

## concate

In [7]:
dfs = []
for dmn in domains:
    print(dmn)
    df = get_meta_data(dmn, nrows=None)
    if set(df.columns)!= KEYS2USE: continue
    df = df[df.category.apply(len)>0]
    if len(df)==0: continue
    print(df.shape)
    df['domain'] = dmn
    display(df.sample(2))
    dfs.append(df)

AMAZON_FASHION
All_Beauty
Appliance
(29639, 6)


,category,description,title,brand,feature,asin,domain
10252,"[Appliances, Parts & Accessories, Refrigerator...",[This radiant glass defrost heater is commonly...,Whirlpool 60106-35 Defrost Heater Assembly,Whirlpool,"[450 watts, 150 volts, Genuine Whirlpool repla...",B00A8O08CS,Appliance
4930,"[Appliances, Parts & Accessories, Range Parts ...","[Part Number WB3X714 replaces 2261, AH249112, ...",GE WB3X714 Gas Surface Burner Knob,GE,[Genuine GE factory part],B003BIGDFE,Appliance


Arts_Crafts_and_Sewi
(284889, 6)


,category,description,title,brand,feature,asin,domain
220310,"[Arts, Crafts & Sewing, Needlework, Needlepoin...","[Once the basics of needle tatting is learned,...","Handy Hands T440 Tatting Book, Pink",Handy Hands,"[Once the basics of needle tatting is learned,...",B00TYO1JDU,Arts_Crafts_and_Sewi
57227,"[Arts, Crafts & Sewing, Sewing, Sewing Machines]",[The SINGER Futura XL-400 is the next generati...,SINGER Futura XL400 Portable Sewing and125 Emb...,SINGER,[SINGER EMBROIDERY SEWING MACHINE: The SINGER ...,B004YEZLVG,Arts_Crafts_and_Sewi


Automotive
(897587, 6)


...

,category,description,title,brand,feature,asin,domain
59239,"[Tools & Home Improvement, Safety & Security, ...","[Protective eyewear for science (classrooms, l...","Neiko 53829A Lab Safety Goggles, Impact and Ch...",Neiko,"[Perfect protective, anti-chemical eyewear for...",B000MFO504,Tools_and_Home_Improvement
144957,"[Tools & Home Improvement, Lighting & Ceiling ...",[A beautiful fruit design is the centerpiece o...,"Z-Lite H20-1-01 Fruit One Light Pendant, Metal...",Z-Lite,[A beautiful fruit design is the centerpiece o...,B00311P9QU,Tools_and_Home_Improvement


Toys_and_Game
(571640, 6)


,category,description,title,brand,feature,asin,domain
49909,"[Toys & Games, Dress Up & Pretend Play, Preten...",[Fake Sesame Dinner Roll. Manufactured using o...,Sesame Dinner Roll Fake Bread,Flora-cal Products,"[Fake Sesame Dinner Roll Soft Touch, Size: 5""L...",B000J58FQ2,Toys_and_Game
116189,"[Toys & Games, Action Figures & Statues, Acces...",[],Legends of Horror Previews Exclusive Shrieker ...,Full Moon,[],B001NTO0F8,Toys_and_Game


Video_Game
(82394, 6)


,category,description,title,brand,feature,asin,domain
10951,"[Video Games, PC, Games, </span></span></span>...",[With the Super Strategy 4 Pack you'll have ho...,Super Strategy 4 Pack Age of Empires Gold Civi...,by\n \n Atari,"[Includes Majesty Gold Edition, Civilization I...",B0000BVGNY,Video_Game
48401,"[Video Games, Nintendo DS, Accessories, </span...",[Nintendo DSi XL Skin Decal Sticker - Pink Hea...,Nintendo DSi XL Skin Decal Sticker - Pink Hearts,by\n \n DecalSkin,"[This 4-piece sticker skin set Cover Front, Ba...",B004CN4UQ6,Video_Game


In [8]:
df = pd.concat(dfs)
df.shape

(13706665, 7)

In [9]:
df.sample(3)

,category,description,title,brand,feature,asin,domain
523732,"[Books, Children's Books, Growing Up &amp; Fac...",[K-Gr 2Three of Jesus's parables from the New ...,"Who Counts?: 100 Sheep, 10 Coins, and 2 Sons",Amy-Jill Levine,[],0664262740,Book
870211,"[Clothing, Shoes & Jewelry, Women]",[The inferno 3 pack no show tab socks employs ...,Saucony Women's 3 Pack Inferno Tab Socks,Saucony,"[95% Polyester, 4% Rubber, 1% Spandex, Importe...",B00CRJP5ZC,Clothing_Shoes_and_Jewelry
339147,"[Electronics, Camera & Photo, Accessories, Cab...",[<br>The dCables&reg; AV Cable for the Konica ...,dCables Konica Minolta DiMAGE Z5 AV Cable - TV...,dCables,[Use this cable to display images from the Kon...,B00AFGWEQW,Electronic


## cleanup

### category

In [10]:
%%time
def try2eval(x):
    try:
        x = eval(x)
        x = (o.replace('|','') for o in x)
        return '|'.join(x)
    except SyntaxError:
        return ""

df.category = df.category.astype(str)
df.category = df.category.apply(try2eval)
print("df.shape:", df.shape)

df.shape: (13706665, 7)
CPU times: user 3min 18s, sys: 5.7 s, total: 3min 24s
Wall time: 3min 23s


### text

In [11]:
import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [12]:
%%time
df.fillna('', inplace=True)

for col in ['description', 'feature']:
    df[col] = df[col].apply(lambda x: "\n".join(x) if len(x) else "")

for col in ['description', 'title', 'brand', 'feature',]:
    df[col] = df[col].apply(cleanhtml)

CPU times: user 3min 13s, sys: 6.03 s, total: 3min 19s
Wall time: 3min 22s


In [ ]:
%%time
df['txt'] = df.title + " " + df.brand + " " + df.description + " " + df.feature

# train val split

In [ ]:
import numpy as np

np.random.seed(101)
df['is_validation'] = np.random.choice(2, size=len(df), p=[0.85,0.15])

In [ ]:
df.is_validation.value_counts()

In [ ]:
df.sample(3)

# save

In [ ]:
%%time
df.to_csv(f'../data/data__{prfx}.csv', index=False)

In [ ]:
ls -hl ../data/data__{prfx}.csv

## make a sample

In [ ]:
import numpy as np
np.random.seed(101)
df_sample = df.sample(10000)

In [ ]:
df_sample.to_csv(f'../data/data_sample__{prfx}.csv', index=False)